### Calculo de errores para los 3 casos
## Errores 
1. Error de manera individual:
  * Puedo calcular $d_{2D}$ usando **geopy.distance.geodesic**.
  * Luego calcular la diferencia de alturas $\Delta z$
  * Finalmente usar un $d_{3D} = \sqrt{(d_{2D})^2 + (\Delta z)^2}$ y así obtendría mi distancia en 3D.
2. ECM
  * Para obtener el Error Cuadrático Medio puedo, sumar los valores y dividir por la cantidad de puntos (100 puntos)
3. RECM(RMSE)
  * Por último se podría tomar la raiz del ECM para expresar el error promedio en metros.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tensorflow.keras.models import load_model
import pickle
from Utils import utils_nn as utlnn
from Utils import geo_conversions as gc

In [2]:
# Carga conjunto de Test.
x_test = pd.read_excel("./Train_Test/Dataset_Separado/x_test_new.xlsx")
y_test = pd.read_excel("./Train_Test/Dataset_Separado/y_test_new.xlsx")

# Quita de columnas no usadas en Train 
x_test = x_test.drop(columns=['latitude_pos_tx', 'longitude_pos_tx', 'elevation_pos_tx', 'year'])

In [3]:
# Cargamos los modelos.
case = 3	# Puede ser 1, 2 o 3, dependiendo del modelo que se quiera cargar.
  
if case == 1: 
	model_x =  load_model('./modelos_entrenamiento/mod_x_R0/modelo1_x_con_scaler/modelo1_x.keras')
	with open('./modelos_entrenamiento/mod_x_R0/modelo1_x_con_scaler/scaler_x1.pkl', 'rb') as f:
		scaler_x = pickle.load(f)

	model_y = load_model('./modelos_entrenamiento/mod_y_R0/modelo1_y_con_scaler/modelo1_y.keras')
	with open('./modelos_entrenamiento/mod_y_R0/modelo1_y_con_scaler/scaler_y1.pkl', 'rb') as f:
		scaler_y = pickle.load(f)
		
	model_z = load_model('./modelos_entrenamiento/mod_z_R0/modelo1/modelo1.keras')
	with open('./modelos_entrenamiento/mod_z_R0/modelo1/scaler_z1.pkl','rb') as f:
		scaler_z = pickle.load(f)

if case == 2: 
    model_z = load_model('./modelos_entrenamiento/mod_z_R0/modelo1/modelo1.keras')
    with open('./modelos_entrenamiento/mod_z_R0/modelo1/scaler_z1.pkl','rb') as f:
        scaler_z = pickle.load(f)

    model_xy = load_model('./modelos_entrenamiento/mod_x_y_R0/modelo1_x_y_con_scaler/mod_x_y_R0.keras')
    with open('./modelos_entrenamiento/mod_x_y_R0/modelo1_x_y_con_scaler/scaler_x.pkl', 'rb') as f:
        scaler_x = pickle.load(f)
    with open('./modelos_entrenamiento/mod_x_y_R0/modelo1_x_y_con_scaler/scaler_y.pkl', 'rb') as f:
        scaler_y = pickle.load(f)

if case == 3:
      model_xyz = load_model('./modelos_entrenamiento/mod_x_y_z_R0/modelo1/modelo1_xyz.keras')
      with open('./modelos_entrenamiento/mod_x_y_z_R0/modelo1/scaler_x.pkl', 'rb') as f:
          scaler_x = pickle.load(f)
      with open('./modelos_entrenamiento/mod_x_y_z_R0/modelo1/scaler_y.pkl', 'rb') as f:
          scaler_y = pickle.load(f)
      with open('./modelos_entrenamiento/mod_x_y_z_R0/modelo1/scaler_z.pkl', 'rb') as f:
          scaler_z = pickle.load(f)

In [ ]:
# Predicción para el conjunto Test completo, descalado de MinMax_scaler y escalado a R0
R0	= 6.371E6
y_pred_x = scaler_x.inverse_transform(model_x.predict(x_test)) * R0
y_pred_y = scaler_y.inverse_transform(model_y.predict(x_test)) * R0
y_pred_z = scaler_z.inverse_transform(model_z.predict(x_test)) * R0

# Obtencion de muestras verdaderas
y_true_x = y_test.iloc[:,  0:100].to_numpy()
y_true_y = y_test.iloc[:,100:200].to_numpy()
y_true_z = y_test.iloc[:,200:300].to_numpy()


33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [5]:
phi_true, theta_true, rho_true	= gc.transform_cartesian_to_spherical(y_true_x, y_true_y, y_true_z)
lat_true, lon_true, alt_true 		= gc.transform_spherical_to_geographic(phi_true, theta_true, rho_true)

phi_pred, theta_pred, rho_pred 	= gc.transform_cartesian_to_spherical(y_pred_x, y_pred_y, y_pred_z)
lat_pred, lon_pred, alt_pred 		= gc.transform_spherical_to_geographic(phi_pred, theta_pred, rho_pred) # Altura en metros

In [18]:
alt_pred[10]-alt_true[10]

array([1.00000000e+00, 7.00056818e+01, 1.18511364e+02, 1.88017045e+02,
       1.77522727e+02, 2.43528409e+02, 3.86090909e+02, 5.37210227e+02,
       6.30329545e+02, 6.81948864e+02, 7.81068182e+02, 8.56187500e+02,
       1.02250923e+03, 1.16183097e+03, 1.25665270e+03, 1.27997443e+03,
       1.33229616e+03, 1.43789666e+03, 1.47627592e+03, 1.43765518e+03,
       1.82403445e+03, 1.90891371e+03, 1.87329297e+03, 2.19689311e+03,
       2.10849325e+03, 2.10909339e+03, 2.32769354e+03, 2.22429368e+03,
       2.26143786e+03, 2.35662607e+03, 2.67731428e+03, 2.65000249e+03,
       2.86269070e+03, 2.65187891e+03, 2.94819638e+03, 2.83651385e+03,
       3.13383132e+03, 2.90514879e+03, 3.08196626e+03, 2.75652876e+03,
       3.11983629e+03, 2.97064382e+03, 3.28045135e+03, 3.32675888e+03,
       3.64756641e+03, 3.75330788e+03, 3.90254936e+03, 4.19429084e+03,
       4.23453232e+03, 3.56277379e+03, 3.95261399e+03, 4.01855291e+03,
       4.25699183e+03, 4.67043075e+03, 4.07586967e+03, 4.37630859e+03,
      

In [6]:
distances_2d = np.array(list(map(lambda args: np.array(utlnn.distances_by_geodesic(*args)), zip(lat_true, lon_true, lat_pred, lon_pred))))
distances_3d = np.array(list(map(lambda args: np.array(utlnn.distances_3D(*args)), zip(distances_2d,alt_true/1E3,alt_pred/1E3))))# ERROR DISTANCIA 2D ESTA EN KM MIENTRAS QUE LAS ALTURAS INGRESAN EN METROS 

In [13]:
error_2d = np.array(list(map(lambda args: np.array(utlnn.ecm_recm(args)),distances_2d)))
error_3d =  np.array(list(map(lambda args: np.array(utlnn.ecm_recm(args)),distances_3d)))

In [22]:
ubi = 50
print(f'Distancia: {distances_2d[ubi].mean()}')
print(f'Error 2D: {error_2d[ubi]}')

distancia_punto = utlnn.distances_by_geodesic(lat_true[ubi], lon_true[ubi], lat_pred[ubi], lon_pred[ubi])
print(f'Latitud True Inicial: {lat_true[0][0]}, Longitud True Inicial: {lon_true[0][0]}')
print(f'Latitud Predicha Inicial: {lat_pred[0][0]}, Longitud Predicha Inicial: {lon_pred[0][0]}')

Distancia: 30.335720540219164
Error 2D: [1444.71587811   38.00941828]
Latitud True Inicial: -42.280000450453734, Longitud True Inicial: -63.40000836722327
Latitud Predicha Inicial: -42.279998779296875, Longitud Predicha Inicial: -63.40000534057617


In [12]:
true_points = np.stack((y_true_x, y_true_y, y_true_z) , axis = 1)
pred_points = np.stack((y_pred_x, y_pred_y, y_pred_z) , axis = 1)

errors = np.linalg.norm(true_points - pred_points, axis = 1)


In [6]:
# Vamos a ver estas Gráficas en Google Earth
from simplekml import Kml
indice = 1
R0 = 6.371E6
latitudes_graf 	= lat_true[indice]
longitudes_graf = lon_true[indice]
altura_graf 	= alt_true[indice] -R0  # Convertir a km

longitudes_graf_pred = lon_pred[indice]
latitudes_graf_pred = lat_pred[indice]
altura_graf_pred = alt_pred[indice]  - R0# Convertir a km

kml = Kml()

linestring = kml.newlinestring(name = 'Trayectoria Verdadera')
linestring.coords = list(zip(longitudes_graf, latitudes_graf, altura_graf	))
linestring.altitudmode = 'absolute'
linestring.extrude = 0
linestring.style.linestyle.color = 'ff0000ff'  # Rojo
linestring.style.linestyle.width = 5

linestring_pred = kml.newlinestring(name = 'Trayectoria Predicha')
linestring_pred.coords = list(zip(longitudes_graf_pred, latitudes_graf_pred, altura_graf_pred	))
linestring_pred.altitudmode = 'absolute'
linestring_pred.extrude = 0
linestring_pred.style.linestyle.color = 'ff00ff00'  # Verde
linestring_pred.style.linestyle.width = 5


kml.save('Trayectoria_Real_Pred.kml')

In [8]:
indice = 100
R0 = 6.371E6
latitudes_graf 	= lat_true[indice]
longitudes_graf = lon_true[indice]
altura_graf 	= alt_true[indice] -R0  # Convertir a km

longitudes_graf_pred = lon_pred[indice]
latitudes_graf_pred = lat_pred[indice]
altura_graf_pred = alt_pred[indice] - R0  # Convertir a km

R0 = 6.371E6
print(altura_graf_pred)

[2.000000e+00 3.475000e+02 5.125000e+02 8.910000e+02 1.264500e+03
 4.140000e+03 7.477000e+03 1.094750e+04 1.378000e+04 1.631550e+04
 2.053800e+04 2.483400e+04 2.784000e+04 3.188450e+04 3.565850e+04
 4.082550e+04 4.429850e+04 4.986000e+04 5.392450e+04 5.896750e+04
 6.591050e+04 6.925200e+04 7.712000e+04 8.000350e+04 8.778200e+04
 9.372400e+04 9.880600e+04 1.033300e+05 1.105855e+05 1.176510e+05
 1.244225e+05 1.317740e+05 1.379365e+05 1.437910e+05 1.500445e+05
 1.596240e+05 1.663315e+05 1.710835e+05 1.799345e+05 1.883960e+05
 1.925745e+05 2.028980e+05 2.063365e+05 2.176425e+05 2.208580e+05
 2.257330e+05 2.317830e+05 2.358720e+05 2.351310e+05 2.376945e+05
 2.353950e+05 2.370205e+05 2.362220e+05 2.301290e+05 2.257995e+05
 2.146465e+05 2.085895e+05 1.996870e+05 1.960225e+05 1.838400e+05
 1.799605e+05 1.706370e+05 1.613180e+05 1.601965e+05 1.479670e+05
 1.351615e+05 1.309055e+05 1.239925e+05 1.202630e+05 1.064085e+05
 9.616550e+04 9.715200e+04 8.985850e+04 8.552500e+04 7.652800e+04
 7.257350e

In [32]:
errors[2].mean()

np.float64(28699.057669202004)

In [50]:
xTrue=true_points[0][0]
yTrue=true_points[0][1]
zTrue=true_points[0][2]

xPred=pred_points[0][0]
yPred=pred_points[0][1]
zPred=pred_points[0][2]
indice = 20
print(f'x_true:{xTrue[indice]}, y_true:{yTrue[indice]}, z_true:{zTrue[indice]}')
print(f'x_pred:{xPred[indice]}, y_pred:{yPred[indice]}, z_pred:{zPred[indice]}')	

x_true:-4206666.829014664, y_true:3826342.731829337, z_true:3008392.031732509
x_pred:-4206549.5, y_pred:3824224.0, z_pred:3011937.75


In [45]:
d =np.sqrt((xTrue-xPred)**2 + (yTrue-yPred)**2)

In [46]:
d.mean()

np.float64(2562.4081003608862)

In [43]:
distances_2d[2].mean()

np.float64(18.790703063211566)